In [ ]:
!pip install wandb 

In [15]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import  Conv2D,MaxPooling2D,Dropout,Flatten,Dense
from tensorflow.keras import Model

from tqdm import tqdm

import wandb
from wandb.keras import WandbCallback

import numpy as np
import cv2

In [ ]:
wandb.init(project="Face Recognition")

# **Load Dataset**

In [ ]:
dataset_path = "/content/drive/MyDrive/Datasets/7-7 dataset"

width = height = 224

idg = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    zoom_range = 0.1,
    brightness_range =  (0.9, 1.1), 
    validation_split = 0.2
)

train_data = idg.flow_from_directory(
    dataset_path,
    target_size = (width , height),
    class_mode = 'categorical',
    subset = 'training'
)

val_data = idg.flow_from_directory(
    dataset_path,
    target_size = (width , height),
    class_mode = 'categorical',
    subset = 'validation'
)

# **Model**

In [18]:
batch_size = 32
epochs = 10

In [19]:
class MahdisNet(Model):
  def __init__(self):
    super().__init__()

    self.Conv2D_1 = Conv2D(32, (3,3), activation = 'relu',input_shape=(width, height, 3))
    self.Conv2D_2 = Conv2D(64, (3,3), activation = 'relu')
    self.MaxPooling = MaxPooling2D()
    self.flatten = Flatten()
    self.dense_1 = Dense(128, activation = 'relu')
    self.dense_2 = Dense(14, activation='softmax')
    self.dropout = Dropout(0.5)

  def call(self, x):
    y = self.Conv2D_1(x)
    z = self.MaxPooling(y)
    j = self.Conv2D_2(z)
    k = self.MaxPooling(j)
    m = self.flatten(k)
    n = self.dense_1(m)
    w = self.dropout(n)
    out = self.dense_2(w)

    return out

In [20]:
model = MahdisNet()

In [21]:
config=wandb.config
config.learning_rate=0.001

In [22]:
optimizer = tf.keras.optimizers.Adam(learning_rate=config.learning_rate)

loss_function = tf.keras.losses.CategoricalCrossentropy()

In [23]:
train_loss = tf.keras.metrics.MeanAbsoluteError()
test_loss = tf.keras.metrics.MeanAbsoluteError()
train_accuracy = tf.keras.metrics.CategoricalAccuracy()
test_accuracy = tf.keras.metrics.CategoricalAccuracy()

In [24]:
#train loop
#.fit()
for epoch in range (epochs):
  train_accuracy.reset_states()
  test_accuracy.reset_states()
  train_loss.reset_states()
  test_loss.reset_states()
  print("Epoch: ", epoch)

  #Training------------------------------
  for i ,(images, labels) in enumerate(tqdm(train_data)):
    if len(train_data) <= i:
            break
    with tf.GradientTape() as gTape:
      predictions = model(images)
      loss = loss_function(labels, predictions)

      train_loss(labels, predictions)
      train_accuracy(labels, predictions)

    #محاسبات مشتق ها
    gradiants = gTape.gradient(loss, model.trainable_variables)
    
    #update w
    optimizer.apply_gradients(zip(gradiants, model.trainable_variables))
  
  #Validation---------------------------
  for i, (images, labels) in enumerate(tqdm(val_data)):
    if len(train_data) <= i:
            break
    predictions = model(images)
    loss = loss_function(labels, predictions)
    test_accuracy(labels, predictions)
    test_loss(labels, predictions)

  print("Train Accuracy: ",train_accuracy.result())
  print("Test Accuracy: ",test_accuracy.result())
  print("Train loss : ", train_loss.result())
  print("Test loss : ", test_loss.result())

  wandb.log({'epochs':  epoch ,
              'Train_loss': np.mean(train_loss.result()),
              'Train_accuracy': float(train_accuracy.result()), 
              'val_loss': np.mean(test_loss.result()),
              'val_accuracy':float(test_accuracy.result())})

Epoch:  0


100%|██████████| 35/35 [00:34<00:00,  1.02it/s]
35it [00:32,  1.07it/s]


Train Accuracy:  tf.Tensor(0.19266056, shape=(), dtype=float32)
Test Accuracy:  tf.Tensor(0.33396226, shape=(), dtype=float32)
Train loss :  tf.Tensor(0.12913331, shape=(), dtype=float32)
Test loss :  tf.Tensor(0.11811438, shape=(), dtype=float32)
Epoch:  1


100%|██████████| 35/35 [00:34<00:00,  1.03it/s]
35it [00:32,  1.06it/s]


Train Accuracy:  tf.Tensor(0.59541285, shape=(), dtype=float32)
Test Accuracy:  tf.Tensor(0.51226413, shape=(), dtype=float32)
Train loss :  tf.Tensor(0.08583041, shape=(), dtype=float32)
Test loss :  tf.Tensor(0.08752284, shape=(), dtype=float32)
Epoch:  2


...

Train Accuracy:  tf.Tensor(0.94587153, shape=(), dtype=float32)
Test Accuracy:  tf.Tensor(0.7641509, shape=(), dtype=float32)
Train loss :  tf.Tensor(0.013856041, shape=(), dtype=float32)
Test loss :  tf.Tensor(0.04420266, shape=(), dtype=float32)
Epoch:  8


100%|██████████| 35/35 [00:33<00:00,  1.05it/s]
35it [00:32,  1.08it/s]


Train Accuracy:  tf.Tensor(0.94220185, shape=(), dtype=float32)
Test Accuracy:  tf.Tensor(0.8037736, shape=(), dtype=float32)
Train loss :  tf.Tensor(0.016296305, shape=(), dtype=float32)
Test loss :  tf.Tensor(0.032151587, shape=(), dtype=float32)
Epoch:  9


100%|██████████| 35/35 [00:34<00:00,  1.01it/s]
35it [00:34,  1.01it/s]

Train Accuracy:  tf.Tensor(0.9752294, shape=(), dtype=float32)
Test Accuracy:  tf.Tensor(0.8462264, shape=(), dtype=float32)
Train loss :  tf.Tensor(0.008704695, shape=(), dtype=float32)
Test loss :  tf.Tensor(0.027487412, shape=(), dtype=float32)


In [27]:
model.save_weights("/content/drive/MyDrive/Datasets/face_recognition.h5")